<a href="https://colab.research.google.com/github/nikhilpalarapu/ML-Projects/blob/main/Brain_tumor_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.metrics import confusion_matrix, roc_curve, auc
import matplotlib.pyplot as plt

In [ ]:
# Define the ablation parameters
resnet50_weights = 'imagenet'
max_conv_layers = 8
validation_split = 0.2

In [ ]:
# Load the images from local storage
train_ds,val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    '/kaggle/input/brain-tumors-dataset/Data',
    validation_split=0.2,
    subset='both',
    seed=123,
    labels='inferred',
    label_mode='binary',
    color_mode='rgb',
    batch_size=32,
    image_size=(224, 224)
)

In [ ]:
# Define the model architecture
model = Sequential()

In [ ]:
# Add VGG16 as a feature extractor
resnet50 = ResNet50(weights=resnet50_weights, include_top=False, input_shape=(224, 224, 3))
model.add(resnet50)

In [ ]:
# Add a few more convolution layers if needed
for i in range(max_conv_layers - len(resnet50.layers)):
    model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu'))

In [ ]:
# Add a flatten layer
model.add(Flatten())

In [ ]:
# Add a dropout layer
model.add(Dropout(0.2))

In [ ]:
# Add a fully connected layer
model.add(Dense(128, activation='relu'))

In [ ]:
# Add a dropout layer
model.add(Dropout(0.2))

In [ ]:
# Add an output layer
model.add(Dense(1, activation='sigmoid'))

In [ ]:
# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Train the model
history=model.fit(train_ds, epochs=3, validation_data=val_ds)

In [ ]:
# Evaluate the model on the validation set
loss, accuracy = model.evaluate(val_ds)

In [ ]:
print('Validation loss:', loss)
print('Validation accuracy:', accuracy)

In [ ]:
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Loss vs. Epoch')
plt.legend()

In [ ]:
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Accuracy vs. Epoch')
plt.legend()
plt.show()

In [ ]:
# Generate ROC and Confusion Matrix
from sklearn.metrics import confusion_matrix, roc_curve, auc
import numpy as np

def plot_roc_curve(fpr, tpr, auc):
    plt.figure()
    plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = {:.2f})'.format(auc))
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic')
    plt.legend(loc='lower right')
    plt.show()

In [ ]:
# Get true labels and predicted probabilities
true_labels = np.concatenate([y for x, y in val_ds], axis=0)
predicted_probs = model.predict(val_ds)

In [ ]:
# Calculate ROC
fpr, tpr, _ = roc_curve(true_labels, predicted_probs)
roc_auc = auc(fpr, tpr)
plot_roc_curve(fpr, tpr, roc_auc)

In [ ]:
# Calculate Confusion Matrix
predicted_labels = (predicted_probs > 0.5).astype(np.int32)
conf_matrix = confusion_matrix(true_labels, predicted_labels)
print('Confusion Matrix:')
print(conf_matrix)